In [1]:
# ===============================================================
#  General L-layer core (SageMath 10.7, Jupyter-friendly)
#
#  Data:
#    L >= 2
#    D = [D1, D2, ..., DL] with D1 < D2 < ... < DL
#    p1 : partition of D2 with D1 parts (list length D1, positive ints)
#    comps : compositions dict for layers 2..L-1
#            comps[I] is a list of length D[I-1] (i.e. length D_I) summing to D_{I+1}
#            Example for L=3: comps = {2: c2}
#            Example for L=4: comps = {2: c2, 3: c3}
#
#  S-sets (recursive generalisation of your L=3 definitions):
#    S^{(L-1)}_{dL} = { j in [D_{L-1}] : #children(j) >= dL }
#    S^{(I)}_{d_{I+1},...,dL} = { i in [D_I] : | children(i) ∩ S^{(I+1)}_{d_{I+2},...,dL} | >= d_{I+1} }
#
#  Monomial exponent list (direct generalisation of your (0.9) pattern):
#    for each d2..dL in the allowed ranges:
#      e_I = d_I - 1 for I >= 2
#      e1 runs 0..|S^{(1)}_{d2..dL}|-1
#
#  Matrix:
#    rows indexed by leaves a_L in [D_L]
#    columns indexed by exponent vectors (e1,...,eL)
#    entry = Π_{I=1..L} ( xI_{a_I(leaf)} )^{eI}
#    where a_I(leaf) is the ancestor in layer I obtained from parent maps.
#
#  Physicist sanity:
#    assert len(exps) == D_L
#    det(M), optionally factor
# ===============================================================

from sage.all import QQ, PolynomialRing, matrix, Graph, show as sage_show
from sage.all import factor, ZZ
from sage.all import SR

# -----------------------------
# helpers
# -----------------------------

def _cumsums(parts):
    s = [0]
    for x in parts:
        s.append(s[-1] + x)
    return s

def _edge_sizes(D, p1, comps):
    """
    Return dict edge_sizes[layer] for edges layer -> layer+1.
    layer=1 uses p1. layer=2..L-1 use comps[layer].
    """
    L = len(D)
    edge = {1: p1}
    for layer in range(2, L):
        if layer not in comps:
            raise ValueError(f"Missing comps[{layer}] for edge {layer}->{layer+1}.")
        edge[layer] = comps[layer]
    return edge

# -----------------------------
# (i) structure: parent + children
# -----------------------------

def build_structure_L(D, p1, comps, minimal_checks=True):
    """
    Build parent maps and children lists for general L.

    Returns:
      parent[layer]  : list of length D[layer] (=D_{layer+1}) with 1-based parent in [1..D_{layer}]
                       for layer=1..L-1
      children[layer]: list (1-based indexed via children[layer][i-1]) of children in layer+1 for i in [D_layer]
    """
    L = len(D)
    edge = _edge_sizes(D, p1, comps)

    if minimal_checks:
        # light shape checks
        if len(p1) != D[0] or sum(p1) != D[1]:
            raise ValueError("p1 must have length D1 and sum to D2.")
        for layer in range(2, L):
            comp = edge[layer]
            if len(comp) != D[layer-1] or sum(comp) != D[layer]:
                raise ValueError(f"comps[{layer}] must have length D{layer} and sum to D{layer+1}.")

    parent = {}
    children = {}

    for layer in range(1, L):
        sizes = edge[layer]                # list length D_layer, sums D_{layer+1}
        D_layer = D[layer-1]
        D_next  = D[layer]
        # parent[layer] maps nodes in layer+1 to nodes in layer
        parent_layer = [None] * D_next
        children_layer = [[] for _ in range(D_layer)]
        s = _cumsums(sizes)
        for i in range(1, D_layer + 1):
            for j in range(s[i-1] + 1, s[i] + 1):
                parent_layer[j-1] = i
                children_layer[i-1].append(j)
        parent[layer] = parent_layer
        children[layer] = children_layer

    return parent, children

def ancestor_chains_L(D, parent):
    """
    For each leaf a_L in [D_L], compute its ancestor chain (a1,...,aL).
    Returns list anc of length D_L with anc[aL-1] = (a1,...,aL).
    """
    L = len(D)
    DL = D[-1]
    anc = [None] * DL
    for leaf in range(1, DL + 1):
        chain = [None] * L
        chain[L-1] = leaf
        # walk up
        for layer in range(L-1, 0, -1):   # layer = L-1,...,1 corresponds to parent[layer]
            chain[layer-1] = parent[layer][chain[layer] - 1]
        anc[leaf - 1] = tuple(chain)
    return anc

# -----------------------------
# (ii) S-sets: recursive definition
# -----------------------------

def S_level(D, children, I, d_vec):
    """
    Compute S^{(I)}_{d_{I+1},...,d_L} as a list of nodes in [D_I].

    Inputs:
      I     : layer index (1..L-1)
      d_vec : list [d_{I+1},...,d_L] of length (L-I)
    """
    L = len(D)
    if not (1 <= I <= L-1):
        raise ValueError("I must be in 1..L-1")
    if len(d_vec) != (L - I):
        raise ValueError("d_vec must have length L-I")

    # Base: I = L-1, d_vec = [dL]
    if I == L - 1:
        dL = d_vec[0]
        out = []
        for j in range(1, D[I-1] + 1):  # nodes in layer L-1
            if len(children[I][j-1]) >= dL:
                out.append(j)
        return out

    # Recursive: compute S^{(I+1)}_{d_{I+2},...,d_L}
    d_next = d_vec[0]
    deeper = d_vec[1:]   # thresholds below I+1
    S_deeper = set(S_level(D, children, I+1, deeper))

    out = []
    for i in range(1, D[I-1] + 1):  # nodes in layer I
        kids = children[I][i-1]     # nodes in layer I+1
        # count how many children lie in S_deeper
        count = 0
        for k in kids:
            if k in S_deeper:
                count += 1
        if count >= d_next:
            out.append(i)
    return out

def S1_dvec(D, p1, comps, d_vec, structure=None):
    """
    Convenience: S^{(1)}_{d2,...,dL}.
    d_vec = [d2,...,dL].
    """
    if structure is None:
        parent, children = build_structure_L(D, p1, comps, minimal_checks=True)
    else:
        parent, children = structure
    return S_level(D, children, I=1, d_vec=d_vec)

# -----------------------------
# (iii) monomial exponent vectors
# -----------------------------

def exponent_vectors_L(D, p1, comps, structure=None, minimal_checks=True):
    """
    General-L exponent vectors (e1,...,eL) using the direct generalisation:
      for each d2..dL:
        eI = dI - 1 for I>=2
        e1 runs 0..|S^{(1)}_{d2..dL}|-1

    Returns list of tuples length L.
    """
    L = len(D)
    if L < 2:
        raise ValueError("Need L>=2")

    if structure is None:
        parent, children = build_structure_L(D, p1, comps, minimal_checks=minimal_checks)
    else:
        parent, children = structure

    edge = _edge_sizes(D, p1, comps)
    max_d = {2: max(edge[1])}   # d2 range from p1
    for I in range(3, L+1):
        max_d[I] = max(edge[I-1])

    exps = []

    # loop over all threshold tuples (d2,...,dL)
    # implemented as nested recursion for cleanliness
    def _recurse_build(I, chosen):
        """
        I is current threshold index (2..L). chosen holds [d2,...,d_{I-1}].
        """
        if I == L + 1:
            d_vec = chosen[:]  # [d2,...,dL]
            k = len(S_level(D, children, I=1, d_vec=d_vec))
            for e1 in range(k):
                e = [None] * L
                e[0] = e1
                for idx, dI in enumerate(d_vec, start=2):
                    e[idx-1] = dI - 1
                exps.append(tuple(e))
            return

        for dI in range(1, max_d[I] + 1):
            chosen.append(dI)
            _recurse_build(I+1, chosen)
            chosen.pop()

    _recurse_build(2, [])

    if minimal_checks:
        assert len(exps) == D[-1], f"Expected {D[-1]} exponent vectors, got {len(exps)}."
    return exps

def monomials_as_strings_L(exps):
    """
    Pretty-print helper: "C1^e1 * ... * CL^eL".
    """
    out = []
    L = len(exps[0]) if exps else 0
    for e in exps:
        out.append(" * ".join([f"C{i}^{e[i-1]}" for i in range(1, L+1)]))
    return out

# -----------------------------
# (iv) ring and variables
# -----------------------------

def build_ring_and_vars_L(D):
    """
    Polynomial ring with global layer variables xI_a:
      x1_1..x1_D1, x2_1..x2_D2, ..., xL_1..xL_DL
    Returns R, x_var[(I,a)].
    """
    L = len(D)
    names = []
    for I in range(1, L+1):
        for a in range(1, D[I-1] + 1):
            names.append(f"x{I}_{a}")
    R = PolynomialRing(QQ, names)
    gens = R.gens()
    name_to_var = {names[k]: gens[k] for k in range(len(names))}
    x_var = {}
    for I in range(1, L+1):
        for a in range(1, D[I-1] + 1):
            x_var[(I,a)] = name_to_var[f"x{I}_{a}"]
    return R, x_var

# -----------------------------
# (v) matrix + determinant
# -----------------------------

def monomial_matrix_L(D, p1, comps, exps=None, structure=None, minimal_checks=True):
    """
    Build DL x DL monomial evaluation matrix for general L.

    Returns: R, M, exps
    """
    if structure is None:
        parent, children = build_structure_L(D, p1, comps, minimal_checks=minimal_checks)
    else:
        parent, children = structure

    if exps is None:
        exps = exponent_vectors_L(D, p1, comps, structure=(parent, children), minimal_checks=minimal_checks)

    if minimal_checks:
        assert len(exps) == D[-1], f"Expected {D[-1]} exponent vectors, got {len(exps)}."

    anc = ancestor_chains_L(D, parent)
    R, x_var = build_ring_and_vars_L(D)

    DL = D[-1]
    M = matrix(R, DL, DL)

    for col, e in enumerate(exps):
        for leaf in range(1, DL + 1):
            chain = anc[leaf - 1]   # (a1,...,aL)
            entry = R(1)
            for I in range(1, len(D)+1):
                entry *= x_var[(I, chain[I-1])] ** e[I-1]
            M[leaf - 1, col] = entry

    return R, M, exps



def monomial_matrix_and_det_L(D, p1, comps,
                              factor_det=False,
                              normalize_diffs=False,
                              **kwargs):
    """
    Return (R, M, detM).
    If factor_det=True: detM is a factorization.
    If normalize_diffs=True (and factor_det=True): orient diff factors as (xI_a - xI_b) with a<b.
    """

    def _parse_xIa(varname):
        if not varname.startswith("x") or "_" not in varname[1:]:
            return None
        I_str, a_str = varname[1:].split("_", 1)
        return (int(I_str), int(a_str))

    def _is_two_var_linear_diff(f):
        # detect f = ±(x - y) with coeffs ±1 and no constant term
        if f.total_degree() != 1:
            return None
        d = f.dict()
        for exp in d:              # ETuple-safe
            if sum(exp) == 0:      # constant term
                return None

        terms = []
        for exp, coeff in d.items():
            if sum(exp) == 1:
                j = next(pos for pos, ej in enumerate(exp) if ej == 1)  # ETuple-safe
                terms.append((j, ZZ(coeff)))
            else:
                return None

        if len(terms) != 2:
            return None
        (j1, c1), (j2, c2) = terms
        if abs(c1) != 1 or abs(c2) != 1 or c1 != -c2:
            return None

        vars_ = f.parent().gens()
        v1, v2 = vars_[j1], vars_[j2]
        sign = 1 if c1 == 1 else -1
        return (v1, v2, sign)

    def _normalize_factorization(fac):
        # Return a SYMBOLIC product so Sage doesn't re-flip signs by refactoring
        expr = SR(fac.unit())

        for f, e in fac:
            parsed = _is_two_var_linear_diff(f)
            if parsed is None:
                expr *= SR(f)**e
                continue

            v1, v2, sign = parsed

            if sign == -1:
                expr *= (-1)**e
            base = v1 - v2

            p1p = _parse_xIa(str(v1))
            p2p = _parse_xIa(str(v2))
            if p1p is not None and p2p is not None and p1p[0] == p2p[0]:
                _, a1 = p1p
                _, a2 = p2p
                if a1 > a2:
                    expr *= (-1)**e
                    base = v2 - v1

            expr *= SR(base)**e

        return expr

    # compute matrix and determinant
    R, M, exps = monomial_matrix_L(D, p1, comps, **kwargs)
    detM = M.det()

    if factor_det:
        detM = detM.factor()
        if normalize_diffs:
            detM = _normalize_factorization(detM)

    return R, M, detM

  




# -----------------------------
# (vi) optional graph plot (general L)
# -----------------------------

def layered_graph_L(D, p1, comps, structure=None, x_spacing=3.0):
    """
    Graph with node labels aligned with determinant variables xI_a.
    Indices increase upwards.
    """
    if structure is None:
        parent, children = build_structure_L(D, p1, comps, minimal_checks=False)
    else:
        parent, children = structure

    L = len(D)
    G = Graph()

    # add vertices
    for I in range(1, L+1):
        for a in range(1, D[I-1] + 1):
            G.add_vertex(f"x{I}_{a}")

    # add edges xI_parent -> x(I+1)_child
    for I in range(1, L):
        for child in range(1, D[I] + 1):
            par = parent[I][child - 1]
            G.add_edge(f"x{I}_{par}", f"x{I+1}_{child}")

    # positions
    pos = {}
    for I in range(1, L+1):
        x = (I-1) * x_spacing
        for a in range(1, D[I-1] + 1):
            pos[f"x{I}_{a}"] = (x, a)

    return G, pos

def show_layered_graph_L(D, p1, comps, structure=None, x_spacing=3.0,
                         vertex_size=60, edge_thickness=0.5, edge_color="lightgray",
                         label_dx=0.15, label_dy=-0.15, fontsize=9,
                         axes=False, frame=False):
    """
    Draw graph with manual labels offset down/right.
    """
    G, pos = layered_graph_L(D, p1, comps, structure=structure, x_spacing=x_spacing)

    P = G.plot(
        pos=pos,
        vertex_size=vertex_size,
        vertex_labels=False,
        edge_thickness=edge_thickness,
        edge_color=edge_color,
        axes=axes,
        frame=frame
    )

    for v in G.vertices():
        x, y = pos[v]
        P += text(
            v,
            (x + label_dx, y + label_dy),
            horizontal_alignment='left',
            vertical_alignment='top',
            fontsize=fontsize
        )

    sage_show(P)
    return G, pos

# -----------------------------
# (vii) driver
# -----------------------------

def build_L(D, p1, comps, display=(), factor_det=False):
    """
    Physicist-friendly driver for general L.
    display can include: "graph", "monomials", "matrix", "det"
    """
    if isinstance(display, str):
        display = (display,)
    else:
        display = tuple(display)

    out = {}
    structure = build_structure_L(D, p1, comps, minimal_checks=True)
    out["structure"] = structure

    exps = exponent_vectors_L(D, p1, comps, structure=structure, minimal_checks=True)
    out["exps"] = exps

    if "monomials" in display:
        out["monomials"] = monomials_as_strings_L(exps)
        print("Monomials:")
        for m in out["monomials"]:
            print(m)

    if ("matrix" in display) or ("det" in display):
        R, M, _ = monomial_matrix_L(D, p1, comps, exps=exps, structure=structure, minimal_checks=True)
        out["R"] = R
        out["M"] = M
        if "matrix" in display:
            print("Matrix M:")
            print(M)
            
    if "det" in display:
        _, _, detM = monomial_matrix_and_det_L(
            D, p1, comps,
            factor_det=factor_det,
            normalize_diffs=True,
            exps=exps,
            structure=structure,
            minimal_checks=True
        )
        out["det"] = detM
        print("det(M):")
        print(out["det"])


    if "graph" in display:
        G, pos = show_layered_graph_L(D, p1, comps, structure=structure, x_spacing=4.5)
        out["G"] = G
        out["pos"] = pos

    return out


from sage.all import ZZ

# ===============================================================
# Conjecture 1 (referred to as ``refined conjecture'' in the code below) checker:
#
# Conjecture (physics form):
#   det(M) = ±  ∏_{I=1..L} ∏_{(a<b) siblings in layer I} (xI_a - xI_b)^{e_{I,a,b}}
#   with ALL sibling pairs present and e_{I,a,b} >= 1.
#
# Notes:
#   - "siblings in layer I" means: same parent in layer I-1 (from compositions),
#     and for I=1 we use the "virtual root" convention: all pairs 1<=a<b<=D1.
#   - We do NOT assume the factorization pattern; we factor the determinant  and then test.
# ===============================================================

def sibling_pairs_by_layer(D, structure, include_layer1_virtual_root=True):
    """
    sib[I] = set of pairs (a,b) with a<b in layer I that share a parent in layer I-1.
    If include_layer1_virtual_root=True: sib[1] = all pairs 1<=a<b<=D1.
    """
    parent, children = structure
    L = len(D)
    sib = {}

    if include_layer1_virtual_root:
        D1 = D[0]
        sib[1] = {(a, b) for a in range(1, D1 + 1) for b in range(a + 1, D1 + 1)}

    for I in range(2, L + 1):
        pairs = set()
        for par in range(1, D[I - 2] + 1):
            kids = children[I - 1][par - 1]  # nodes in layer I
            for i in range(len(kids)):
                for j in range(i + 1, len(kids)):
                    a, b = kids[i], kids[j]
                    pairs.add((a, b) if a < b else (b, a))
        sib[I] = pairs

    return sib


def parse_xIa(varname):
    # "x3_10" -> (3,10)
    if not varname.startswith("x"):
        return None
    rest = varname[1:]
    if "_" not in rest:
        return None
    I_str, a_str = rest.split("_", 1)
    return (int(I_str), int(a_str))


def is_linear_diff_factor(f):
    """
    Detect f = ±(x - y) with coeffs ±1 and no constant term.
    Returns (v1, v2, sign) meaning f = sign*(v1 - v2), or None.
    ETuple-safe.
    """
    if f.total_degree() != 1:
        return None

    d = f.dict()

    # reject constant term
    for exp in d:
        if sum(exp) == 0:
            return None

    terms = []
    for exp, coeff in d.items():
        if sum(exp) == 1:
            j = next(pos for pos, ej in enumerate(exp) if ej == 1)  # ETuple-safe
            terms.append((j, ZZ(coeff)))
        else:
            return None

    if len(terms) != 2:
        return None

    (j1, c1), (j2, c2) = terms
    if abs(c1) != 1 or abs(c2) != 1 or c1 != -c2:
        return None

    vars_ = f.parent().gens()
    v1, v2 = vars_[j1], vars_[j2]
    sign = 1 if c1 == 1 else -1
    return (v1, v2, sign)


def canonicalize_pair_from_factor(f):
    """
    If f is a linear diff ±(xI_a - xI_b), return:
      (I, a, b, sign_flips)
    where a<b is enforced, and sign_flips is 0 or 1 indicating whether
    we had to multiply by -1 to achieve the canonical (a<b) orientation
    *after* rewriting f to (v1 - v2).
    Returns None if not a linear diff in xI_a vars or if cross-layer.
    """
    parsed = is_linear_diff_factor(f)
    if parsed is None:
        return None

    v1, v2, sign = parsed  # f = sign*(v1 - v2)
    pA = parse_xIa(str(v1))
    pB = parse_xIa(str(v2))
    if pA is None or pB is None:
        return None

    I1, a = pA
    I2, b = pB
    if I1 != I2:
        return None

    # First normalize to (v1 - v2) by absorbing `sign` separately:
    # sign = -1 means f = -(v1 - v2), i.e. one sign flip.
    sign_flips = 1 if sign == -1 else 0

    # Now enforce a<b: if we swap (a,b), that's another sign flip.
    if a < b:
        return (I1, a, b, sign_flips)
    else:
        return (I1, b, a, sign_flips + 1)


def canonical_unit_after_orienting_linear_diffs(fac):
    """
    Canonical unit after orienting each linear diff to (xI_a - xI_b) with a<b.
    """
    unit = ZZ(fac.unit())
    for f, e in fac:
        info = canonicalize_pair_from_factor(f)
        if info is None:
            continue
        I, a, b, flips = info
        # each flip contributes (-1)^e
        if flips % 2 == 1:
            unit *= (-1) ** int(e)
    return unit


def test_refined_conjecture(D, p1, comps,
                            include_layer1_virtual_root=True,
                            verbose=True,
                            show_first_missing=20):
    """
    Refined conjecture check:
      (i) Every factor in det(M).factor() is a linear diff ±(xI_a - xI_b)
          and the pair (a,b) is an allowed sibling pair (virtual root at layer1 if enabled).
      (ii) Completeness: every allowed sibling pair appears with exponent >= 1.
      (iii) We report canonical unit (after orienting diffs), but do not require it to be +1.

    Returns dict with ok flags and diagnostics.
    """
    structure = build_structure_L(D, p1, comps, minimal_checks=True)
    sib = sibling_pairs_by_layer(D, structure, include_layer1_virtual_root=include_layer1_virtual_root)

    # build and factor determinant
    R, M, exps = monomial_matrix_L(D, p1, comps, structure=structure, minimal_checks=True)
    fac = M.det().factor()

    # exponent map for canonical pairs
    exp_map = {}   # key (I,a,b) -> exponent (positive int)
    bad_factors = []  # list (f, e, reason)

    for f, e in fac:
        info = canonicalize_pair_from_factor(f)
        if info is None:
            bad_factors.append((f, e, "not a linear ±(xI_a-xI_b) within one layer"))
            continue

        I, a, b, flips = info  # canonical a<b
        if I not in sib or (a, b) not in sib[I]:
            bad_factors.append((f, e, f"linear diff but NOT an allowed sibling pair in layer {I}: ({a},{b})"))
            continue

        exp_map[(I, a, b)] = exp_map.get((I, a, b), 0) + int(e)

    # completeness: every allowed sibling pair appears with exponent >= 1
    missing = {}  # layer -> list of missing pairs
    for I in sorted(sib.keys()):
        miss_I = []
        for (a, b) in sorted(sib[I]):
            if exp_map.get((I, a, b), 0) <= 0:
                miss_I.append((a, b))
        if miss_I:
            missing[I] = miss_I

    ok_support = (len(bad_factors) == 0)
    ok_complete = (len(missing) == 0)
    ok = ok_support and ok_complete

    canon_unit = canonical_unit_after_orienting_linear_diffs(fac)

    if verbose:
        print("=== Refined conjecture test: ALL sibling pairs appear with positive exponent ===")
        print(f"D={D}, p1={p1}, comps={comps}")
        print(f"canonical unit (after orienting diffs) = {canon_unit}")
        print(f"#factors (from Sage factor) = {len(list(fac))}")
        print(f"bad factors (non-sibling or non-linear-diff) = {len(bad_factors)}")
        print(f"missing sibling pairs = {sum(len(v) for v in missing.values())}")

        if bad_factors:
            print("\n-- Bad factors (first few) --")
            for f, e, reason in bad_factors[:10]:
                print(f"  ({f})^{e} :: {reason}")

        if missing:
            print("\n-- Missing sibling pairs (showing up to first few per layer) --")
            for I in sorted(missing):
                lst = missing[I]
                preview = lst[:show_first_missing]
                print(f"  layer {I}: missing {len(lst)} pairs; first {len(preview)} = {preview}")

    return {
        "ok": ok,
        "ok_support": ok_support,
        "ok_complete": ok_complete,
        "canonical_unit": canon_unit,
        "bad_factors": bad_factors,
        "missing": missing,
        "exp_map": exp_map,
        "factorization": fac
    }

# --------------------------
# Usage (fixed D, p1, comps)
# --------------------------
# D = [2,4]
# p1 = [2,2]
# comps = {}
# res = test_refined_conjecture(D, p1, comps)

In [2]:
D = [2,4]
p1 = [2,2]
comps = { }
res = test_refined_conjecture(D, p1, comps)

=== Refined conjecture test: ALL sibling pairs appear with positive exponent ===
D=[2, 4], p1=[2, 2], comps={}
canonical unit (after orienting diffs) = -1
#factors (from Sage factor) = 3
bad factors (non-sibling or non-linear-diff) = 0
missing sibling pairs = 0


In [3]:
D = [2,4,7]
p1 = [2,2]
comps = { 2: [2,1,2,2] }
res = test_refined_conjecture(D, p1, comps)

=== Refined conjecture test: ALL sibling pairs appear with positive exponent ===
D=[2, 4, 7], p1=[2, 2], comps={2: [2, 1, 2, 2]}
canonical unit (after orienting diffs) = 1
#factors (from Sage factor) = 6
bad factors (non-sibling or non-linear-diff) = 0
missing sibling pairs = 0


In [10]:
D = [2, 4, 7, 12 ]
p1 = [2, 2]
comps = {2: [2, 1, 2, 2] , 3: [2,3,1,2,2,1,1  ] } 
res = test_refined_conjecture(D, p1, comps)

=== Refined conjecture test: ALL sibling pairs appear with positive exponent ===
D=[2, 4, 7, 12], p1=[2, 2], comps={2: [2, 1, 2, 2], 3: [2, 3, 1, 2, 2, 1, 1]}
canonical unit (after orienting diffs) = -1
#factors (from Sage factor) = 12
bad factors (non-sibling or non-linear-diff) = 0
missing sibling pairs = 0


In [4]:
# ===============================================================
# Sweep refined conjecture at fixed D
#
# Requires: your core code (build_structure_L, monomial_matrix_L, etc.)
#           and the refined checker: test_refined_conjecture(...)
#
# This block is SELF-CONTAINED for enumeration of p1 and comps.
# ===============================================================

from itertools import product

# --------------------------
# Enumerators
# --------------------------

def compositions_positive(n, k):
    """
    All compositions of n into k positive integers (order matters).
    Returns lists of length k summing to n.
    """
    if k <= 0:
        return
    if k == 1:
        if n >= 1:
            yield [n]
        return
    # first part ranges 1..n-(k-1)
    for first in range(1, n - (k - 1) + 1):
        for rest in compositions_positive(n - first, k - 1):
            yield [first] + rest


def partitions_fixed_length_sum(n, k, nonincreasing=True):
    """
    Partitions of n into k positive parts.
    If nonincreasing=True, parts are returned in weakly decreasing order.
    """
    if k <= 0:
        return
    if k == 1:
        if n >= 1:
            yield [n]
        return

    # recursive generator enforcing nonincreasing order: a1 >= a2 >= ... >= ak >= 1
    def _rec(remaining, parts_left, max_part, acc):
        if parts_left == 1:
            if 1 <= remaining <= max_part:
                yield acc + [remaining]
            return
        # choose next part between 1 and min(max_part, remaining-(parts_left-1))
        hi = min(max_part, remaining - (parts_left - 1))
        for a in range(hi, 0, -1):
            yield from _rec(remaining - a, parts_left - 1, a, acc + [a])

    if nonincreasing:
        yield from _rec(n, k, n, [])
    else:
        # if you ever want all k-tuples summing to n without ordering constraint
        yield from compositions_positive(n, k)


# --------------------------
# Sweep driver
# --------------------------

def sweep_refined_conjecture_fixed_D(
    D,
    stop_after=None,
    verbose_every=50,
    include_layer1_virtual_root=True,
    show_first_failures=5
):
    """
    Sweep over all (p1, comps) compatible with fixed D and test refined conjecture.

    D = [D1, D2, ..., DL]
    p1: partitions of D2 with D1 parts (weakly decreasing)
    comps[layer]: compositions of D_{layer+1} with D_layer parts (positive)

    Returns dict with summary and counterexamples.
    """
    L = len(D)
    if L < 2:
        raise ValueError("Need L >= 2")

    # enumerate p1 (as partitions)
    p1_list = list(partitions_fixed_length_sum(D[1], D[0], nonincreasing=True))

    # enumerate comps for layers 2..L-1
    comps_lists = {}  # layer -> list of compositions
    for layer in range(2, L):
        comps_lists[layer] = list(compositions_positive(D[layer], D[layer-1]))

    layers = list(range(2, L))
    total = len(p1_list)
    for layer in layers:
        total *= len(comps_lists[layer])

    tested = 0
    failures = []   # list of dicts with p1, comps, bad_factors, missing, canonical_unit

    # iterate
    for p1 in p1_list:
        if layers:
            for comp_tuple in product(*[comps_lists[layer] for layer in layers]):
                comps = {layer: list(comp_tuple[i]) for i, layer in enumerate(layers)}

                tested += 1
                if verbose_every and tested % verbose_every == 0:
                    print(f"[{tested}/{total}] ...")

                res = test_refined_conjecture(
                    D, p1, comps,
                    include_layer1_virtual_root=include_layer1_virtual_root,
                    verbose=False
                )

                if not res["ok"]:
                    failures.append({
                        "p1": tuple(p1),
                        "comps": {k: tuple(v) for k, v in comps.items()},
                        "ok_support": res["ok_support"],
                        "ok_complete": res["ok_complete"],
                        "bad_factors": res["bad_factors"],
                        "missing": res["missing"],
                        "canonical_unit": res["canonical_unit"],
                    })

                if stop_after is not None and tested >= stop_after:
                    break
        else:
            # L=2, no comps
            comps = {}
            tested += 1
            res = test_refined_conjecture(
                D, p1, comps,
                include_layer1_virtual_root=include_layer1_virtual_root,
                verbose=False
            )
            if not res["ok"]:
                failures.append({
                    "p1": tuple(p1),
                    "comps": {},
                    "ok_support": res["ok_support"],
                    "ok_complete": res["ok_complete"],
                    "bad_factors": res["bad_factors"],
                    "missing": res["missing"],
                    "canonical_unit": res["canonical_unit"],
                })
            if stop_after is not None and tested >= stop_after:
                break

        if stop_after is not None and tested >= stop_after:
            break

    # summary
    print("=== Sweep summary (refined conjecture) ===")
    print(f"D = {D}")
    print(f"tested = {tested} (out of total {total})")
    print(f"failures = {len(failures)}")

    if failures:
        print("\nFirst few failures:")
        for f in failures[:show_first_failures]:
            print(f"p1={f['p1']}, comps={f['comps']}, unit={f['canonical_unit']}, "
                  f"ok_support={f['ok_support']}, ok_complete={f['ok_complete']}")
            if f["bad_factors"]:
                print("  bad_factors (first 3):", f["bad_factors"][:3])
            if f["missing"]:
                # show first missing layer/pairs
                I0 = sorted(f["missing"].keys())[0]
                print(f"  missing layer {I0} (first 10):", f["missing"][I0][:10])

    return {
        "D": D,
        "tested": tested,
        "total": total,
        "failures": failures
    }


# --------------------------
# Usage examples
# --------------------------
# Small quick sweep:
# summary = sweep_refined_conjecture_fixed_D([2,4], verbose_every=10)
#
# Slightly bigger, but still manageable:
# summary = sweep_refined_conjecture_fixed_D([2,4,7], stop_after=200, verbose_every=20)
#
# If you want full sweep and it is still small enough:
# summary = sweep_refined_conjecture_fixed_D([2,4,7], verbose_every=50)


In [5]:
summary = sweep_refined_conjecture_fixed_D([2,4], verbose_every=10)

=== Sweep summary (refined conjecture) ===
D = [2, 4]
tested = 2 (out of total 2)
failures = 0


In [6]:
summary = sweep_refined_conjecture_fixed_D([2,4,7], stop_after=200, verbose_every=20)

[20/40] ...
[40/40] ...
=== Sweep summary (refined conjecture) ===
D = [2, 4, 7]
tested = 40 (out of total 40)
failures = 0


In [8]:
sweep_refined_conjecture_fixed_D([2,4,7])

=== Sweep summary (refined conjecture) ===
D = [2, 4, 7]
tested = 40 (out of total 40)
failures = 0


{'D': [2, 4, 7], 'tested': 40, 'total': 40, 'failures': []}

In [12]:
sweep_refined_conjecture_fixed_D([2,5,9])

[50/140] ...
[100/140] ...
=== Sweep summary (refined conjecture) ===
D = [2, 5, 9]
tested = 140 (out of total 140)
failures = 0


{'D': [2, 5, 9], 'tested': 140, 'total': 140, 'failures': []}

In [8]:
summary = sweep_refined_conjecture_fixed_D([2,5,8], stop_after=200, verbose_every=20)

[20/70] ...
[40/70] ...
[60/70] ...
=== Sweep summary (refined conjecture) ===
D = [2, 5, 8]
tested = 70 (out of total 70)
failures = 0


In [17]:
summary = sweep_refined_conjecture_fixed_D([2,6,9], stop_after=200, verbose_every=20)

[20/168] ...
[40/168] ...
[60/168] ...
[80/168] ...
[100/168] ...
[120/168] ...
[140/168] ...
[160/168] ...
=== Sweep summary (refined conjecture) ===
D = [2, 6, 9]
tested = 168 (out of total 168)
failures = 0


In [7]:
summary = sweep_refined_conjecture_fixed_D([2,6,10], stop_after=200, verbose_every=20)

[20/378] ...
[40/378] ...
[60/378] ...
[80/378] ...
[100/378] ...
[120/378] ...
[140/378] ...
[160/378] ...
[180/378] ...
[200/378] ...
=== Sweep summary (refined conjecture) ===
D = [2, 6, 10]
tested = 200 (out of total 378)
failures = 0
